# Building a Custom Retrieval Augmented (RAG) Agent with Langchain and Metal

In this example, we'll create a simple RAG agent that can answer questions based on information from a PDF file. [Langchain]('https://www.langchain.com/') will assist us in integrating additional tools like a calculator or a search engine, enhancing our application's ability to reason and perform complex computations. Metal will streamline the entire process by processing the documents, breaking them into smaller parts, and extracting their meaning so we can have engaging conversations with our data.

Let's start by running `pip install metal_sdk` in your terminal and setting up the Metal Client. 

In [7]:
from metal_sdk.metal import Metal
import os

api_key= os.environ['METAL_API_KEY']
client_id= os.environ['METAL_CLIENT_ID']

metal = Metal(api_key, client_id)


Creating a Retrieval pipeline (RAG) in Metal is a [3 step process]('https://docs.getmetal.io/guides/misc-retrieval-pipeline').

1. First, we want to create a [Data Source]('https://docs.getmetal.io/rest-api/datasources/create'), this is the repository where our files will be pre processed. 

2. Then connect it to an [Index]('https://docs.getmetal.io/rest-api/indexes/post').  This where our preprocessed data will be transformed into embeddings and made queryable for our agent. 

3. Finally, upload the files as [Data Entities]('https://docs.getmetal.io/rest-api/dataentities/create') to our datasource. 

### Step 1: Create a Data Source

In [8]:
datasource_payload = {
    "name": "Food Safety Datasource",
    "sourcetype": "File",
    "autoExtract": True,
    "metadataFields": []
}

datasource = metal.add_datasource(datasource_payload)


### Step 2: Connect Index to Data Source

In [9]:
# Connect Index to the Datasource
datasource_id = datasource['data']['id']

index_payload = {
    "model": "text-embedding-ada-002",
    "name": "Food Safety Index",
    "datasource": datasource_id,
    "indexType": "HNSW",
    "dimensions": 1536
}

food_report_index = metal.add_index(index_payload)


### Step 3: Upload Files (Data Entities) to the Data Source

 For this example, we will use [The State of Food Security and Nutrition in the World 2022 - UNICEF DATA]('https://data.unicef.org/resources/sofi-2021/')

In [10]:
# Add files (Data Entities) to the Datasource
file_path = 'data/SOFI2021_InBrief.pdf'
results = metal.add_data_entity(datasource_id, file_path)


### Step 4: Create Question Answering Chain


Now, let's add the Index Id to our Metal Client and let's initialize the Question Answering (QA) chain using the MetalRetriever module from Langchain. 

In [11]:
#Instiantiate Metal as the Retriever including the index_id
from langchain.retrievers.metal import MetalRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

index_id = food_report_index['data']['id']
metal = Metal(api_key, client_id, index_id)

retriever = MetalRetriever(client=metal,params={"limit": 2} )

qa_chain = RetrievalQA.from_chain_type(
  llm=OpenAI(temperature=0),
  chain_type="stuff",
  retriever=retriever,
)


### Step 5: Querying the Retriever


In [12]:
query = "What are the the strategies to lower the cost of nutritious foods?"
response = qa_chain(query)
result = response["result"]
print(result)


 Some strategies to lower the cost of nutritious foods include buying in bulk, shopping at discount stores, growing your own food, and taking advantage of coupons and sales.


### Step 5: Empowering the Application with Agency


Here, we define a set of tools for our agent, including the "Food Security Report," which utilizes the QA chain for answering questions, the "Math" tool for mathematical inquiries, and the "Search" tool for searching the internet for relevant information. Each tool is accompanied by a custom name and a descriptive explanation, helping our agent make informed decisions.

In [13]:
from langchain.chat_models import ChatOpenAI

# chat completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0,
)


In [14]:
from langchain.agents import load_tools, Tool

#defining the tools for the agent
tools_chain = load_tools(["llm-math","serpapi"], llm=llm)

#defining the tools for the agent
tools = [
    Tool(
        name = "Food Security Report",
        func=qa_chain.run,
        description="use this as the primary source of context information when you are asked the question. Always search for the answers using this tool first, don't make up answers yourself"
    ),

    Tool(
        name = "Math",
        func = tools_chain[0].func,
        description = "use this tool to answer math questions"
        ),


    Tool(
        name = "Search",
        func = tools_chain[1].func,
        description = "use this tool to search questions on the internet"
        )
]


### Step 6: Initialize the Agent Chain


In [15]:
from langchain.agents import initialize_agent

agent_chain = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True,
                         max_iterations=5)


In [62]:
query   = "What is the cost of a healthy diet in Middle Africa? What does that value represent today in Japanese Yen? And what is the root square of this value?"

agent_chain.run(input=query)




> Entering new  chain...
I should start by checking the Food Insecurity Report to find the cost of a healthy diet in Middle Africa. Then I can convert that value to Japanese Yen and calculate the square root.
Action: Food Insecurity Report
Action Input: Cost of a healthy diet in Middle Africa
Observation: The cost of a healthy diet in Middle Africa is 3.81 USD per person per day.
Thought:Now I need to convert 3.81 USD to Japanese Yen. I can use the Search tool to find the current exchange rate.
Action: Search
Action Input: "USD to JPY exchange rate"
Observation: Convert US Dollar to Japanese Yen ; 1 USD, 149.746 JPY ; 5 USD, 748.728 JPY ; 10 USD, 1,497.46 JPY ; 25 USD, 3,743.64 JPY.
Thought:I can see that the exchange rate is 1 USD to 149.746 JPY. Now I can calculate the cost of a healthy diet in Middle Africa in Japanese Yen.
Action: Math
Action Input: 3.81 USD * 149.746 JPY/USD
Observation: Answer: 570.5322600000001
Thought:Now I have the cost of a healthy diet in Middle Africa in 

'The cost of a healthy diet in Middle Africa is 570.53 Japanese Yen today, and the square root of this value is approximately 23.89.'

Our agent performs a range of tasks to arrive at a comprehensive answer. It retrieves relevant data from the vector store, conducts internet searches, and employs mathematical calculations to provide an accurate response.